In [4]:
import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)

In [5]:

kite = KiteConnect(api_key="z8j06t7svnaindta")

# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.

In [6]:
kite.login_url()

'https://kite.zerodha.com/connect/login?api_key=z8j06t7svnaindta&v=3'

In [ ]:
data = kite.generate_session("request_token_here", api_secret="your_secret")
kite.set_access_token(data["access_token"])

In [55]:
def get_chunked_data(ticker_symbol):
    chunks = []
    current_start = start_date
    
    while current_start < end_date:
        # Use smaller chunks (5 days) to avoid data gaps
        current_end = min(current_start + timedelta(days=5), end_date)
        try:
            ticker = yf.Ticker(ticker_symbol)
            chunk = ticker.history(
                start=current_start.strftime("%Y-%m-%d"),
                end=current_end.strftime("%Y-%m-%d"),
                interval="1m",
                prepost=True  # Include pre and post market data
            )
            if not chunk.empty:
                # Convert timezone to IST for both indices
                chunk.index = chunk.index.tz_convert('Asia/Kolkata')
                chunks.append(chunk)
        except Exception as e:
            print(f"Error fetching data for {ticker_symbol} from {current_start} to {current_end}: {e}")
        
        current_start = current_end
    
    if not chunks:
        raise ValueError(f"No data retrieved for {ticker_symbol}")
        
    return pd.concat(chunks).sort_index().drop_duplicates()


In [56]:
print(f"Fetching NIFTY data from {start_date.date()} to {end_date.date()}")
nifty_df = get_chunked_data("^NSEI")
nifty_df.to_csv('nifty_1min_data-new.csv')

# Get India VIX data
print(f"Fetching India VIX data from {start_date.date()} to {end_date.date()}")
vix_df = get_chunked_data("^INDIAVIX")
vix_df.to_csv('vix_1min_data-new.csv')

print(f"Data saved successfully")
print(f"NIFTY records: {len(nifty_df)}")
print(f"India VIX records: {len(vix_df)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7123 entries, 0 to 7122
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Datetime      7123 non-null   object 
 1   Open          7123 non-null   float64
 2   High          7123 non-null   float64
 3   Low           7123 non-null   float64
 4   Close         7123 non-null   float64
 5   Volume        7123 non-null   int64  
 6   Dividends     7123 non-null   float64
 7   Stock Splits  7123 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 445.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7112 entries, 0 to 7111
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Datetime      7112 non-null   object 
 1   Open          7112 non-null   float64
 2   High          7112 non-null   float64
 3   Low           7112 non-null   float64
 4   Close         7112 non-null   f